In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\noised_stake_data.csv')
df_sp.head()

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,116.704707,10.789589,178.524402,0.960872,0.522011,0.543095
1,1,250,18,3867.0,113.487625,5.006354,152.118696,0.773263,0.420089,0.437056
2,2,263,13,3867.0,114.203133,4.979968,100.791271,0.709409,0.385399,0.400965
3,3,273,10,3868.0,116.584423,10.297328,72.470200,0.663096,0.360239,0.374789
4,4,298,25,3868.0,109.178897,0.539409,130.864899,0.478961,0.260204,0.270714


In [2]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Era5Land\Parameters_1000.csv')
df_model1_param.head()

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.098285,-0.328110,0.505590,4.286001,17.206201,0.437698,3.702612,11.623785,0.444232,0.427410
1,1,1.196568,-0.287907,0.504975,5.024441,12.488065,0.393736,4.370584,6.361340,0.350399,0.396229
2,2,1.111208,-0.328610,0.533966,4.615500,14.941232,0.454123,3.942050,9.816933,0.422546,0.450480
3,3,1.043782,-0.375285,0.481698,4.650498,15.363376,0.426325,3.825529,11.479997,0.446073,0.422991
4,4,1.101841,-0.327316,0.478549,4.659066,14.514407,0.408834,4.055419,7.732787,0.362618,0.408789


In [3]:
data = {'DDF':df_model1_param['DDF Model 3']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['d0'] = df_model1_param['d0 Model 3']
df_all_iterations['a'] = df_model1_param['a Model 3']

MSEs = list(df_model1_param['MSE Model 3'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 3']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations.head()

,DDF,d0,a,MSE,Weight
0,3.702612,11.623785,0.444232,0.427410,0.652196
1,4.370584,6.361340,0.350399,0.396229,0.672852
2,3.942050,9.816933,0.422546,0.450480,0.637322
3,3.825529,11.479997,0.446073,0.422991,0.655084
4,4.055419,7.732787,0.362618,0.408789,0.664455


In [4]:
print(min(weight), max(weight))
count = 0
for i in weight:
    if i <= 0.655:
        count +=1
print('count: ', count) 
# there is no zero weight, number of non zero weights is 1000

0.6024414197848518 0.7165982123903413
count:  263


In [5]:
df_all_iterations['DDF*Weight'] = df_all_iterations['DDF']*df_all_iterations['Weight']
df_all_iterations['d0*Weight'] = df_all_iterations['d0']*df_all_iterations['Weight']
df_all_iterations['a*Weight'] = df_all_iterations['a']*df_all_iterations['Weight']

df_all_iterations['Weight*(DDF - xBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['DDF'] - sum(df_all_iterations['DDF*Weight']/1000) )**2
df_all_iterations['Weight*(d0 - yBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['d0'] - sum(df_all_iterations['d0*Weight']/1000) )**2
df_all_iterations['Weight*(a - zBARw)^2'] = df_all_iterations['Weight']*(df_all_iterations['a'] - sum(df_all_iterations['a*Weight']/1000) )**2

df_all_iterations.head()

,DDF,d0,a,MSE,Weight,DDF*Weight,d0*Weight,a*Weight,Weight*(DDF - xBARw)^2,Weight*(d0 - yBARw)^2,Weight*(a - zBARw)^2
0,3.702612,11.623785,0.444232,0.427410,0.652196,2.414828,7.580985,0.289726,0.647713,22.732368,0.021997
1,4.370584,6.361340,0.350399,0.396229,0.672852,2.940758,4.280244,0.235767,1.864244,0.276789,0.005428
2,3.942050,9.816933,0.422546,0.450480,0.637322,2.512357,6.256552,0.269298,0.973627,10.697568,0.016719
3,3.825529,11.479997,0.446073,0.422991,0.655084,2.506044,7.520365,0.292215,0.820967,21.734387,0.022540
4,4.055419,7.732787,0.362618,0.408789,0.664455,2.694642,5.138086,0.240943,1.209828,2.692015,0.006918


In [6]:
ddf_variability = (sum(df_all_iterations['Weight*(DDF - xBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


d0_variability = (sum(df_all_iterations['Weight*(d0 - yBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

a_variability = (sum(df_all_iterations['Weight*(a - zBARw)^2'])*(1000)/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['DDF'][MSEs.index(min(MSEs))], df_all_iterations['d0'][MSEs.index(min(MSEs))], df_all_iterations['a'][MSEs.index(min(MSEs))])
print(round(ddf_variability,3), round(d0_variability,3), round(a_variability,3))
print('RMSE =',min(MSEs)**0.5)

5.235351263645969 3.82658528010333 0.31858938837505096
0.044 0.113 0.004
RMSE = 0.5772694073701696
